In [3]:
database = "T:/Occurrence_Records/EBD_TEST_1.sqlite"
outFile = "T:/Occurrence_Records/EBD_TEST_1.shp"
footprints = False

In [5]:
from datetime import datetime
import sqlite3
import pandas as pd
import geopandas as gpd
timestamp = datetime.now()

# Get the record coordinates as a data frame
records = (pd.read_sql("""SELECT * FROM occurrence_records;""",
                       con=sqlite3.connect(database))
                      .astype({'decimalLongitude': 'float',
                               'decimalLatitude': 'float',
                               'radius_meters': 'float'}))

In [ ]:
# If user just wants shapefile of given coordinates
if footprints == False:
    # Make a point geometry from coordinates
    gdf = gpd.GeoDataFrame(records,
                           geometry=gpd.points_from_xy(records['decimalLongitude'],
                                                       records['decimalLatitude']))

    # Set the coordinate reference system
    gdf.crs={'init' :'epsg:4326'}
    
    # Save shapefile
    gdf.to_file(outFile)

In [ ]:
# Save polygons (footprints)
if footprints == True:
    # Reproject states and record coordinates to facilitate buffering
    footprints = gdf.to_crs(epsg=5070)

    # Point-radius method
    footprints['footprint']=footprints.apply(lambda x: x.geometry.buffer(x.radius_meters), axis=1)
    footprints.set_geometry(col='footprint', inplace=True, drop=True)

    # Shape method


    footprints.to_file(outFile)

print("Exported shapefile: " + str(datetime.now() - timestamp))